In [21]:
import pickle
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
import preprocessing_functions
import feature_engineering_functions
import pycaret
#from pycaret.classification import *
from scipy.stats import pearsonr
from dataprep.eda import create_report
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler

Basic preprocessing actions (Fitbit)
1. Select experiment dates
2. Drop duplicates
3. Type conversion
4. One hot encoding

In [22]:
fitbit=pd.read_pickle('data/daily_fitbit_df_unprocessed.pkl')
fitbit=preprocessing_functions.fitbit_basic_preprocessing(fitbit)
fitbit=preprocessing_functions.fitbit_one_hot_encoding(fitbit)
fitbit

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,...,Martial Arts,Run,Spinning,Sport,Swim,Treadmill,Walk,Weights,Workout,Yoga/Pilates
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,621e301e67b776a240608a72,2021-05-24,34.710232,NaN,NaN,NaN,NaN,74.0,0.700000,0.700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,621e301367b776a24057738e,2021-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,621e360b67b776a24039709f,2021-05-24,NaN,68.321,31.413,NaN,15.4,69.0,0.633333,0.650,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,621e2fce67b776a240279baa,2021-05-24,34.783747,83.170,15.941,96.6,15.2,70.0,0.733333,0.725,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4959,621e345267b776a240691064,2022-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4960,621e310d67b776a24003096d,2022-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4961,621e324e67b776a2400191cb,2022-01-17,33.104768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4962,621e30f467b776a240f22944,2022-01-17,34.352490,NaN,NaN,NaN,13.4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Basic preprocessing actions (Sema)
1. Group positive and negative feelings

In [23]:
sema=pd.read_pickle('data/semas_read_from_the_base_experiment_dates.pkl')
sema = sema.sort_values(by='date', ascending=True)
sema=preprocessing_functions.sema_basic_preprocessing(sema)
sema

,id,date,negative_feelings,positive_feelings
0,621e2f3967b776a240c654db,2021-05-24,0,1
11,621e362467b776a2404ad513,2021-05-24,0,1
10,621e301e67b776a240608a72,2021-05-24,0,1
9,621e30e467b776a240e817c7,2021-05-24,1,0
8,621e36f967b776a240e5e7c9,2021-05-24,1,0
...,...,...,...,...
4932,621e30f467b776a240f22944,2022-01-16,0,1
4946,621e2f9167b776a240011ccb,2022-01-17,1,0
4944,621e346f67b776a24081744f,2022-01-17,1,0
4945,621e30f467b776a240f22944,2022-01-17,0,1


In [24]:
# merge fitbit and sema
data = pd.merge(fitbit, sema, on=['date','id'], how='inner')
data

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,...,Spinning,Sport,Swim,Treadmill,Walk,Weights,Workout,Yoga/Pilates,negative_feelings,positive_feelings
0,621e301e67b776a240608a72,2021-05-24,34.710232,NaN,NaN,NaN,NaN,74.0,0.700000,0.700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
1,621e362467b776a2404ad513,2021-05-24,34.526732,67.118,37.722,95.3,10.6,0.0,0.000000,0.000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,1
2,621e362467b776a2404ad513,2021-05-24,34.526732,67.118,37.722,95.3,10.6,0.0,0.000000,0.000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,1
3,621e362467b776a2404ad513,2021-05-24,34.526732,67.118,37.722,95.3,10.6,0.0,0.000000,0.000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,1
4,621e2f3967b776a240c654db,2021-05-24,NaN,69.820,28.594,NaN,16.4,76.0,0.800000,0.850,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4940,621e366567b776a24076a727,2022-01-16,33.277004,46.697,62.157,96.4,13.0,76.0,0.633333,0.825,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
4941,621e346f67b776a24081744f,2022-01-17,32.903545,71.863,25.475,NaN,18.2,72.0,0.833333,0.625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
4942,621e331067b776a24085dd3f,2022-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
4943,621e2f9167b776a240011ccb,2022-01-17,34.716690,58.489,28.329,96.5,14.2,76.0,0.800000,0.600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0


In [25]:
data = data.drop(columns=['positive_feelings'])
data

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,...,Run,Spinning,Sport,Swim,Treadmill,Walk,Weights,Workout,Yoga/Pilates,negative_feelings
0,621e301e67b776a240608a72,2021-05-24,34.710232,NaN,NaN,NaN,NaN,74.0,0.700000,0.700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,621e362467b776a2404ad513,2021-05-24,34.526732,67.118,37.722,95.3,10.6,0.0,0.000000,0.000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
2,621e362467b776a2404ad513,2021-05-24,34.526732,67.118,37.722,95.3,10.6,0.0,0.000000,0.000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
3,621e362467b776a2404ad513,2021-05-24,34.526732,67.118,37.722,95.3,10.6,0.0,0.000000,0.000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
4,621e2f3967b776a240c654db,2021-05-24,NaN,69.820,28.594,NaN,16.4,76.0,0.800000,0.850,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4940,621e366567b776a24076a727,2022-01-16,33.277004,46.697,62.157,96.4,13.0,76.0,0.633333,0.825,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4941,621e346f67b776a24081744f,2022-01-17,32.903545,71.863,25.475,NaN,18.2,72.0,0.833333,0.625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4942,621e331067b776a24085dd3f,2022-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4943,621e2f9167b776a240011ccb,2022-01-17,34.716690,58.489,28.329,96.5,14.2,76.0,0.800000,0.600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


Preprocessing steps added:
1. drop 'spo2', 'scl_avg'
2. replace outliers and NaNs with columns median
3. date engineering

In [26]:
df = preprocessing_functions.post_preprocessing(data)
df

,id,nightly_temperature,nremhr,rmssd,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,daily_temperature_variation,...,Yoga/Pilates,negative_feelings,month_sin,weekday_sin,week_sin,day_sin,month_cos,weekday_cos,week_cos,day_cos
0,621e301e67b776a240608a72,34.710232,61.6455,36.408,14.4,74.0,0.700000,0.700,0.833333,-1.822489,...,0.0,0,-0.866025,0.000000,0.309017,-0.988468,0.5,1.00000,0.951057,0.151428
1,621e362467b776a2404ad513,34.526732,67.1180,37.722,10.6,0.0,0.000000,0.000,0.000000,-3.007565,...,0.0,0,-0.866025,0.000000,0.309017,-0.988468,0.5,1.00000,0.951057,0.151428
2,621e362467b776a2404ad513,34.526732,67.1180,37.722,10.6,0.0,0.000000,0.000,0.000000,-3.007565,...,0.0,0,-0.866025,0.000000,0.309017,-0.988468,0.5,1.00000,0.951057,0.151428
3,621e362467b776a2404ad513,34.526732,67.1180,37.722,10.6,0.0,0.000000,0.000,0.000000,-3.007565,...,0.0,0,-0.866025,0.000000,0.309017,-0.988468,0.5,1.00000,0.951057,0.151428
4,621e2f3967b776a240c654db,33.980583,69.8200,28.594,16.4,76.0,0.800000,0.850,0.600000,-1.184004,...,0.0,0,-0.866025,0.000000,0.309017,-0.988468,0.5,1.00000,0.951057,0.151428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4940,621e366567b776a24076a727,33.277004,46.6970,62.157,13.0,76.0,0.633333,0.825,0.800000,-2.266232,...,0.0,1,0.866025,-0.781831,0.587785,-0.101168,0.5,0.62349,0.809017,-0.994869
4941,621e346f67b776a24081744f,32.903545,71.8630,25.475,18.2,72.0,0.833333,0.625,0.733333,-3.234348,...,0.0,1,0.866025,0.000000,0.809017,-0.299363,0.5,1.00000,0.587785,-0.954139
4942,621e331067b776a24085dd3f,33.980583,61.6455,36.408,14.4,76.0,0.766667,0.725,0.766667,-1.404268,...,0.0,0,0.866025,0.000000,0.809017,-0.299363,0.5,1.00000,0.587785,-0.954139
4943,621e2f9167b776a240011ccb,34.716690,58.4890,28.329,14.2,76.0,0.800000,0.600,0.933333,-2.096381,...,0.0,1,0.866025,0.000000,0.809017,-0.299363,0.5,1.00000,0.587785,-0.954139


In [27]:
# Because of way too many missing values in spo2 (80%) and scl_avg (95%), I drop these 2 columns
df = data.drop(columns=['spo2', 'scl_avg'])

In [28]:
# Day-related feature extraction
df = preprocessing_functions.date_engineering(df)
df

,id,nightly_temperature,nremhr,rmssd,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,daily_temperature_variation,...,Yoga/Pilates,negative_feelings,month_sin,weekday_sin,week_sin,day_sin,month_cos,weekday_cos,week_cos,day_cos
0,621e301e67b776a240608a72,34.710232,NaN,NaN,NaN,74.0,0.700000,0.700,0.833333,-1.822489,...,NaN,0,-0.866025,0.000000,0.309017,-0.988468,0.5,1.00000,0.951057,0.151428
1,621e362467b776a2404ad513,34.526732,67.118,37.722,10.6,0.0,0.000000,0.000,0.000000,-3.007565,...,0.0,0,-0.866025,0.000000,0.309017,-0.988468,0.5,1.00000,0.951057,0.151428
2,621e362467b776a2404ad513,34.526732,67.118,37.722,10.6,0.0,0.000000,0.000,0.000000,-3.007565,...,0.0,0,-0.866025,0.000000,0.309017,-0.988468,0.5,1.00000,0.951057,0.151428
3,621e362467b776a2404ad513,34.526732,67.118,37.722,10.6,0.0,0.000000,0.000,0.000000,-3.007565,...,0.0,0,-0.866025,0.000000,0.309017,-0.988468,0.5,1.00000,0.951057,0.151428
4,621e2f3967b776a240c654db,NaN,69.820,28.594,16.4,76.0,0.800000,0.850,0.600000,-1.184004,...,0.0,0,-0.866025,0.000000,0.309017,-0.988468,0.5,1.00000,0.951057,0.151428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4940,621e366567b776a24076a727,33.277004,46.697,62.157,13.0,76.0,0.633333,0.825,0.800000,-2.266232,...,NaN,1,0.866025,-0.781831,0.587785,-0.101168,0.5,0.62349,0.809017,-0.994869
4941,621e346f67b776a24081744f,32.903545,71.863,25.475,18.2,72.0,0.833333,0.625,0.733333,-3.234348,...,NaN,1,0.866025,0.000000,0.809017,-0.299363,0.5,1.00000,0.587785,-0.954139
4942,621e331067b776a24085dd3f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0.866025,0.000000,0.809017,-0.299363,0.5,1.00000,0.587785,-0.954139
4943,621e2f9167b776a240011ccb,34.716690,58.489,28.329,14.2,76.0,0.800000,0.600,0.933333,-2.096381,...,NaN,1,0.866025,0.000000,0.809017,-0.299363,0.5,1.00000,0.587785,-0.954139


In [29]:
#create_report(df).show_browser()

In [30]:
# Replace outliers
# separately for each column in the dataframe

columns = df.iloc[:, 1:].columns # excludes id column

for col in columns:
    df[col]=df[col].mask(df[col].sub(df[col].mean()).div(df[col].std()).abs().gt(3))

In [31]:
#create_report(df).show_browser()

In [32]:
# Replace NaN values
# separately for each column in the dataframe

for col in columns:
    df[col] = df[col].apply(pd.to_numeric, errors='coerce')
    df[col] = df[col].fillna(df[col].median())

df

,id,nightly_temperature,nremhr,rmssd,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,daily_temperature_variation,...,Yoga/Pilates,negative_feelings,month_sin,weekday_sin,week_sin,day_sin,month_cos,weekday_cos,week_cos,day_cos
0,621e301e67b776a240608a72,34.710232,61.6455,36.408,14.4,74.0,0.700000,0.700,0.833333,-1.822489,...,0.0,0,-0.866025,0.000000,0.309017,-0.988468,0.5,1.00000,0.951057,0.151428
1,621e362467b776a2404ad513,34.526732,67.1180,37.722,10.6,0.0,0.000000,0.000,0.000000,-3.007565,...,0.0,0,-0.866025,0.000000,0.309017,-0.988468,0.5,1.00000,0.951057,0.151428
2,621e362467b776a2404ad513,34.526732,67.1180,37.722,10.6,0.0,0.000000,0.000,0.000000,-3.007565,...,0.0,0,-0.866025,0.000000,0.309017,-0.988468,0.5,1.00000,0.951057,0.151428
3,621e362467b776a2404ad513,34.526732,67.1180,37.722,10.6,0.0,0.000000,0.000,0.000000,-3.007565,...,0.0,0,-0.866025,0.000000,0.309017,-0.988468,0.5,1.00000,0.951057,0.151428
4,621e2f3967b776a240c654db,33.980583,69.8200,28.594,16.4,76.0,0.800000,0.850,0.600000,-1.184004,...,0.0,0,-0.866025,0.000000,0.309017,-0.988468,0.5,1.00000,0.951057,0.151428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4940,621e366567b776a24076a727,33.277004,46.6970,62.157,13.0,76.0,0.633333,0.825,0.800000,-2.266232,...,0.0,1,0.866025,-0.781831,0.587785,-0.101168,0.5,0.62349,0.809017,-0.994869
4941,621e346f67b776a24081744f,32.903545,71.8630,25.475,18.2,72.0,0.833333,0.625,0.733333,-3.234348,...,0.0,1,0.866025,0.000000,0.809017,-0.299363,0.5,1.00000,0.587785,-0.954139
4942,621e331067b776a24085dd3f,33.980583,61.6455,36.408,14.4,76.0,0.766667,0.725,0.766667,-1.404268,...,0.0,0,0.866025,0.000000,0.809017,-0.299363,0.5,1.00000,0.587785,-0.954139
4943,621e2f9167b776a240011ccb,34.716690,58.4890,28.329,14.2,76.0,0.800000,0.600,0.933333,-2.096381,...,0.0,1,0.866025,0.000000,0.809017,-0.299363,0.5,1.00000,0.587785,-0.954139


In [33]:
# create_report(df).show_browser()